# Random Forest for Human Activity Recognition using the Capture-24 dataset
## Introduction
Here is the implementation of a Random Forest model for Human Activity Recognition (HAR) using the Capture-24 dataset.
## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, log_loss, ConfusionMatrixDisplay
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split
from tqdm import tqdm

## Data Preparation
Here define some functions, that will help us later.

### map_labels
This function will map our labels to a set of labels. The possible set of labels can be chosen from the annotation-label-dictionary. This is useful, as the original labels can be too specific.

### extract_windows
This function will extract a window from the original continuous data. This is necessary, as we want fixed-size windows for our classification task. Here different sizes can be chosen, we will use 10s windows.


In [ ]:
anno_label_dict = pd.read_csv('data/annotation-label-dictionary.csv',
                              index_col='annotation', dtype='string')
print("Annotation-Label Dictionary")
print(anno_label_dict)

"Source: https://github.com/OxWearables/capture24"


# Map the original labels to a set of labels.
def map_labels(data, anno_label_dict, label_mapping):
    data['label'] = (anno_label_dict[label_mapping]
                     .reindex(data['annotation'])
                     .to_numpy())
    return data


# Extract a specific-sized window.
def extract_windows(data, winsize=10):
    X, Y = [], []
    for t, w in data.resample(f'{winsize}s', origin='start'):

        # Check window has no NaNs and is of correct length
        # 10s @ 100Hz = 1000 ticks
        if w.isna().any().any() or len(w) != winsize * 100:
            continue

        x = w[['x', 'y', 'z']].to_numpy()
        y = w['label'].mode(dropna=False).item()

        X.append(x)
        Y.append(y)

    X = np.stack(X)
    Y = np.stack(Y)

    return X, Y


### Feature Extraction

Here we define a function that will extract features from a window of data. The features are commonly used in Human Activity Recognition (HAR) tasks. The features are:
- Minimum, 25th percentile, median, 75th percentile, maximum of x, y, z
- Correlation between x, y, z
- 1s autocorrelation
- Angular features: roll, pitch, yaw
- Spectral entropy, 1st and 2nd dominant frequencies
- Peak features: number of peaks, peak prominence
These features are the usual features used in HAR tasks.

In [ ]:
"Source: https://github.com/OxWearables/capture24"


def extract_features(xyz, sample_rate=100):
    """Extract commonly used HAR time-series features. xyz is a window of shape (N,3)"""

    feats = {}

    x, y, z = xyz.T

    feats['xmin'], feats['xq25'], feats['xmed'], feats['xq75'], feats['xmax'] = np.quantile(x, (0, .25, .5, .75, 1))
    feats['ymin'], feats['yq25'], feats['ymed'], feats['yq75'], feats['ymax'] = np.quantile(y, (0, .25, .5, .75, 1))
    feats['zmin'], feats['zq25'], feats['zmed'], feats['zq75'], feats['zmax'] = np.quantile(z, (0, .25, .5, .75, 1))

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        # xy, xy, zx correlation
        feats['xycorr'] = np.nan_to_num(np.corrcoef(x, y)[0, 1])
        feats['yzcorr'] = np.nan_to_num(np.corrcoef(y, z)[0, 1])
        feats['zxcorr'] = np.nan_to_num(np.corrcoef(z, x)[0, 1])

    v = np.linalg.norm(xyz, axis=1)

    feats['min'], feats['q25'], feats['med'], feats['q75'], feats['max'] = np.quantile(v, (0, .25, .5, .75, 1))

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        # 1s autocorrelation
        feats['corr1s'] = np.nan_to_num(np.corrcoef(v[:-sample_rate], v[sample_rate:]))[0, 1]

    # Angular features
    feats.update(angular_features(xyz, sample_rate))

    # Spectral features
    feats.update(spectral_features(v, sample_rate))

    # Peak features
    feats.update(peak_features(v, sample_rate))

    return feats


def spectral_features(v, sample_rate):
    """ Spectral entropy, 1st & 2nd dominant frequencies """

    feats = {}

    # Spectrum using Welch's method with 3s segment length
    # First run without detrending to get the true spectrum
    freqs, powers = signal.welch(v, fs=sample_rate,
                                 nperseg=3 * sample_rate,
                                 noverlap=2 * sample_rate,
                                 detrend=False,
                                 average='median')

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        feats['pentropy'] = np.nan_to_num(stats.entropy(powers + 1e-16))

    # Spectrum using Welch's method with 3s segment length
    # Now do detrend to focus on the relevant freqs
    freqs, powers = signal.welch(v, fs=sample_rate,
                                 nperseg=3 * sample_rate,
                                 noverlap=2 * sample_rate,
                                 detrend='constant',
                                 average='median')

    peaks, _ = signal.find_peaks(powers)
    peak_powers = powers[peaks]
    peak_freqs = freqs[peaks]
    peak_ranks = np.argsort(peak_powers)[::-1]
    if len(peaks) >= 2:
        feats['f1'] = peak_freqs[peak_ranks[0]]
        feats['f2'] = peak_freqs[peak_ranks[1]]
        feats['p1'] = peak_powers[peak_ranks[0]]
        feats['p2'] = peak_powers[peak_ranks[1]]
    elif len(peaks) == 1:
        feats['f1'] = feats['f2'] = peak_freqs[peak_ranks[0]]
        feats['p1'] = feats['p2'] = peak_powers[peak_ranks[0]]
    else:
        feats['f1'] = feats['f2'] = 0
        feats['p1'] = feats['p2'] = 0

    return feats


def peak_features(v, sample_rate):
    """ Features of the signal peaks. A proxy to step counts. """

    feats = {}
    u = butterfilt(v, (.6, 5), fs=sample_rate)
    peaks, peak_props = signal.find_peaks(u, distance=0.2 * sample_rate, prominence=0.25)
    feats['numPeaks'] = len(peaks)
    if len(peak_props['prominences']) > 0:
        feats['peakPromin'] = np.median(peak_props['prominences'])
    else:
        feats['peakPromin'] = 0

    return feats


def angular_features(xyz, sample_rate):
    """ Roll, pitch, yaw.
    Hip and Wrist Accelerometer Algorithms for Free-Living Behavior
    Classification, Ellis et al.
    """

    feats = {}

    # Raw angles
    x, y, z = xyz.T

    roll = np.arctan2(y, z)
    pitch = np.arctan2(x, z)
    yaw = np.arctan2(y, x)

    feats['avgroll'] = np.mean(roll)
    feats['avgpitch'] = np.mean(pitch)
    feats['avgyaw'] = np.mean(yaw)
    feats['sdroll'] = np.std(roll)
    feats['sdpitch'] = np.std(pitch)
    feats['sdyaw'] = np.std(yaw)

    # Gravity angles
    xyz = butterfilt(xyz, 0.5, fs=sample_rate)

    x, y, z = xyz.T

    roll = np.arctan2(y, z)
    pitch = np.arctan2(x, z)
    yaw = np.arctan2(y, x)

    feats['rollg'] = np.mean(roll)
    feats['pitchg'] = np.mean(pitch)
    feats['yawg'] = np.mean(yaw)

    return feats

# Butterworth filter
def butterfilt(x, cutoffs, fs, order=10, axis=0):
    nyq = 0.5 * fs
    if isinstance(cutoffs, tuple):
        hicut, lowcut = cutoffs
        if hicut > 0:
            btype = 'bandpass'
            Wn = (hicut / nyq, lowcut / nyq)
        else:
            btype = 'low'
            Wn = lowcut / nyq
    else:
        btype = 'low'
        Wn = cutoffs / nyq
    sos = signal.butter(order, Wn, btype=btype, analog=False, output='sos')
    y = signal.sosfiltfilt(sos, x, axis=axis)
    return y


def get_feature_names():
    """ Hacky way to get the list of feature names """

    feats = extract_features(np.zeros((1000, 3)), 100)
    return list(feats.keys())


## Train the model

### Load the data

In [ ]:
# Check the contents of the data directory
print('Content of data/')
print(sorted(os.listdir('data/')))

# Define the label mapping
label_mapping = "label:Willetts2018"

# Load the metadata file
metadata = pd.read_csv('data/metadata.csv')

# Load and concatenate data from all files
data_list = []
for file in sorted(os.listdir('data/')):
    if not file.endswith('csv.gz'):
        continue

    data = pd.read_csv(f'data/{file}', index_col='time', parse_dates=['time'],
                       dtype={'x': 'f4', 'y': 'f4', 'z': 'f4', 'annotation': 'string'})

    file_id = file.split('.')[0]
    print('Current file:', file_id)

    # Find the metadata for this file
    file_metadata = metadata[metadata['pid'] == file_id]

    # Add metadata to the data
    if not file_metadata.empty:
        for col in ['age', 'sex']:
            if col in file_metadata.columns:
                data[col] = file_metadata.iloc[0][col]

    # Map the label
    map_labels(data, anno_label_dict, label_mapping)
    data_list.append(data)

X = []
Y = []

for data in data_list:
    X_, Y_ = extract_windows(data)
    X.append(X_)
    Y.append(Y_)

X_feats = []

for X_ in X:
    X_feats.append(pd.DataFrame([extract_features(x) for x in X_]))

X_feats = pd.concat(X_feats, ignore_index=True)
Y = np.concatenate(Y, axis=0)

print('Data preprocessing done.\n')

### Split the data
Here we split the data into training and evaluation sets. We will use a 90/10 split.

In [ ]:
# Split the data into training and evaluation sets (90% train, 10% eval)
X_train, X_eval, Y_train, Y_eval = train_test_split(X_feats, Y, test_size=0.1, random_state=0)

# Preprocess the features
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_eval_scaled = scaler.transform(X_eval)

### Defining the model
Here we define the model. The hyperparameters are chosen based on a previous grid search. As the dataset is very imbalanced (especially regarding sleep labels) to prevent overfitting, we use a BalancedRandomForestClassifier.

In [ ]:
# Define the model
clf = BalancedRandomForestClassifier(random_state=0, max_depth=None, max_features='sqrt', n_estimators=300,
                                     min_samples_leaf=1, min_samples_split=2, sampling_strategy='all', replacement=True, bootstrap=False)

### Cross-validation
Here we perform K-fold cross-validation to evaluate the model. We will use 5 folds.
As the dataset is relatively big and the training can take a long time, we tqdm to show a progress bar.

In [ ]:
# Cross-validation with progress bar
cv_scores = []
for _ in tqdm(range(5), desc="Cross-validation progress"):  # Progress bar for CV
    score = cross_val_score(clf, X_train_scaled, Y_train, cv=5, scoring='accuracy')
    cv_scores.append(score)

cv_scores = np.array(cv_scores)

# Print the results
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean()}')
print(f'Standard deviation of cross-validation score: {cv_scores.std()}')

### Evaluate the model
Here we evaluate the model on the evaluation dataset.
We calculate the following metrics:
- Accuracy
- F1 score
- Precision
- Recall
- Log loss
- Confusion matrix

In [ ]:
# Fit the model on the training dataset
clf.fit(X_train_scaled, Y_train)

# Evaluate on the last dataset
X_eval_scaled = scaler.transform(X_eval)
Y_pred = clf.predict(X_eval_scaled)

# Calculate the important metrics
accuracy = accuracy_score(Y_eval, Y_pred)
f1_score = f1_score(Y_eval, Y_pred, average='weighted')
precision = precision_score(Y_eval, Y_pred, average='weighted')
recall = recall_score(Y_eval, Y_pred, average='weighted')
log_loss = log_loss(Y_eval, Y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1 score: {clf.score(X_eval_scaled, Y_eval)}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Log loss: {log_loss}')
print(f'Confusion matrix: {clf.classes_}')

disp = ConfusionMatrixDisplay.from_predictions(
    Y_eval,
    Y_pred,
    display_labels=clf.classes_
)
disp.ax_.set_title("Confusion matrix")

print("Confusion matrix")
print(disp.confusion_matrix)

plt.show()